## 유사한 얼굴끼리 군집화 하기 
1. 얼굴 Feature Vector 값 로드 
2. 군집화 수 K결정 → 
$$ K \approx  \sqrt{\frac{N}{2}} $$
3. K-Means로 군집화
4. 군집화 결과 보기 
5. Centroid와 가장 유사한 얼굴 찾아 보기  : 중심 얼굴
6. K의 값을 변경해 가면 적절한 값을 찾기 

In [3]:
## 1. 얼굴 Feature Vector 값 로드 

import pandas as pd 
landmark_ds = pd.DataFrame.from_csv('/Users/goodvc/data/fastcampus/week2/resource/landmark.csv')
landmark_ds.head()

,face_height,eye_to_eye,eyebrow_width,face_width,eye_width,mouth_width,nose_width,lip_to_nose,lip_to_chin
김수현,1,0.31702,0.06413,0.17002,0.45107,0.22673,0.19088,0.20541,0.25671
아기1,1,0.99459,0.05920,0.46626,1.39357,0.59198,0.56177,0.22142,0.19154
양정길1,1,0.43714,0.06665,0.23459,0.61196,0.34220,0.27378,0.20647,0.18553
양정길2,1,0.66480,0.06520,0.35598,0.94308,0.45293,0.40625,0.20946,0.21260
양정길3,1,0.57416,0.06836,0.28739,0.80375,0.41902,0.35779,0.20500,0.17990


In [11]:
## 2. 군집화 수 K결정 sqrt(N/2)
import numpy as np

print ("K=",(np.sqrt(landmark_ds.count()[0]/2)))

K= 2.44948974278


In [42]:
from sklearn.cluster import KMeans
X = landmark_ds.values
cluster = KMeans(n_clusters=3)
cluster.fit(X)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=3, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [36]:
## cluster 결과 
pd.DataFrame( [[a,b] for a,b in zip(landmark_ds.index, cluster.labels_)], columns=['name','group-id'] )\
    .sort(['group-id'])

,name,group-id
0,김수현,0
2,양정길1,0
6,원빈,0
9,최규민1,0
10,최규민2,0
11,최규민3,0
3,양정길2,1
4,양정길3,1
5,와이프1,1
7,전명훈1,1


In [41]:
landmark_ds.index

Index(['김수현', '아기1', '양정길1', '양정길2', '양정길3', '와이프1', '원빈', '전명훈1', '전명훈2', '최규민1', '최규민2', '최규민3'], dtype='object')

In [43]:
landmark_ds

,face_height,eye_to_eye,eyebrow_width,face_width,eye_width,mouth_width,nose_width,lip_to_nose,lip_to_chin
김수현,1,0.31702,0.06413,0.17002,0.45107,0.22673,0.19088,0.20541,0.25671
아기1,1,0.99459,0.05920,0.46626,1.39357,0.59198,0.56177,0.22142,0.19154
양정길1,1,0.43714,0.06665,0.23459,0.61196,0.34220,0.27378,0.20647,0.18553
양정길2,1,0.66480,0.06520,0.35598,0.94308,0.45293,0.40625,0.20946,0.21260
양정길3,1,0.57416,0.06836,0.28739,0.80375,0.41902,0.35779,0.20500,0.17990
와이프1,1,0.65048,0.05942,0.28301,0.91300,0.53354,0.42445,0.19950,0.23461
원빈,1,0.41797,0.06405,0.25496,0.59976,0.32625,0.24684,0.19109,0.22216
전명훈1,1,0.55759,0.06410,0.35101,0.78914,0.47134,0.37391,0.21267,0.25398
전명훈2,1,0.59702,0.06601,0.36339,0.85318,0.55601,0.41339,0.16890,0.26133
최규민1,1,0.36695,0.04336,0.19333,0.51308,0.33244,0.23368,0.17470,0.24359


In [49]:
from sklearn.cluster import AgglomerativeClustering
X = landmark_ds.values
cluster = AgglomerativeClustering(n_clusters=4)
cluster.fit(X)

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward',
            memory=Memory(cachedir=None), n_clusters=4, n_components=None,
            pooling_func=<function mean at 0x105188048>)

In [50]:
## cluster 결과 
pd.DataFrame( [[a,b] for a,b in zip(landmark_ds.index, cluster.labels_)], columns=['name','group-id'] )\
    .sort(['group-id'])

,name,group-id
3,양정길2,0
4,양정길3,0
5,와이프1,0
7,전명훈1,0
8,전명훈2,0
0,김수현,1
9,최규민1,1
10,최규민2,1
11,최규민3,1
1,아기1,2
